<a href="https://colab.research.google.com/github/JoeHasell/test/blob/main/Fiona's_maternal_mortality_notebook_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
R.version.string

print(installed.packages())

In [ ]:
install.packages("janitor") 

library(readxl)
library(dplyr)
library(janitor)
library(tidyr)
library(ggplot2)
date <- Sys.Date()

I'VE MADE A CHANGE HERE!

Download data from Gapminder and save it locally

In [ ]:
#@title
dir.create("data/input", recursive = TRUE)
download.file('https://www.gapminder.org/documentation/documentation/gapdata010.xls', paste0('data/input/',date,'_gapminder_data.xls'))

Read in the Gapminder data

In [ ]:

df <- read_xls(paste0('data/input/',date,'_gapminder_data.xls'), col_names = TRUE)
df <- df[-(1:16),]
head(df)

The first three year ranges seem to be wrong as in another source (https://docs.google.com/spreadsheets/u/0/d/14ZtQy9kd0pMRKWg_zKsTg3qKHoGtflj-Ekal9gIPZ4A/pub?gid=1#) they cover only 10 years rather than 110. I will replace them manually here with the middle of each decade, as is used in the other data from the same source.

There are also two cases where the year for Finland has been entered wrongly three times, there are two 1772s, 1775s & 1967s, the second of each should be 1872, 1875 and 1967 respectively.

There are also two errors for New Zealand, and one error for both Swecen and the US.

In [ ]:
df$year[1:3] <- c(1875, 1885,1895)
df['year'][df$year == '1772' & df$Country == 'Finland' & df$`Maternal deaths`== 487,] <- '1872'
df['year'][df$year == '1775' & df$Country == 'Finland' & df$`Maternal deaths` == 629,] <- '1875'
df['year'][df$year == '1967' & df$Country == 'Finland' & df$`Maternal deaths` == 77,] <- '1957'
df['year'][df$year == '1967' & df$Country == 'Sweden' & df$`Maternal deaths` == 39,] <- '1957'
df['year'][df$year == '1967' & df$Country == 'United States' & df$`Maternal deaths` == 1766.28,] <- '1957'
df['year'][df$year =='1989-02'& df$Country == 'New Zealand',] <- '1889-02'
#Dropping an erroneous row for NZ in 1950 - there are two values but the first is incorrect. You can using the maternal deaths and live births columns. 
df <- df[-which(df$year == '1950' & df$Country == 'New Zealand' & df$MMR == 90),]

For some rows a range of years is given for a particular maternal mortality rate. We use this function to find the mid-value of years given.

In [ ]:
mid_year <- function(year) {
  if(grepl("-", year)){
    year_split <-  unlist(strsplit(year, split = "-"))
    year_begin <- year_split[1]
    year_end_len <- nchar(year_split[2])
    year_end_pref <- 4 - year_end_len
    
    if (year_end_pref > 0) {
      year_pref <- substr(year_begin, 1, year_end_pref)
      year_end <- as.numeric(paste(c(year_pref, year_split[2]),collapse = ""))
      year_out <- round(mean(c(as.numeric(year_begin), year_end)))
    }else{
      year_out <- round(mean(as.numeric(c(year_begin,year_split[2]))))
    }
  }else{
    year_out <- year
  }
  
  return(year_out)
}
year_out_all <- unlist(lapply(df$year, mid_year))
df$mid_year <- as.numeric(year_out_all)

Let's check the mid-year estimates have worked as expected:

In [ ]:
df %>% select(year, mid_year) %>% filter(grepl("-",year)) %>% distinct()

Keeping only the columns we want to take forward.

In [ ]:
df <- df %>% 
  select(entity = Country, year = mid_year, maternal_mortality_rate = MMR)